<a href="https://colab.research.google.com/github/hochschule-pforzheim/project-st23-team-f23/blob/main/3_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dateien ablegen, um sie in Sitzungsspeicher hochzuladen
Laufwerk
83.52 GB verfügbar
Datensatz vorbereiten - Dummies, Skalen
Im ersten Schritt wird der Datensatz aus dem ersten Notebook unter dem Namen 'maindf' geladen. Hier muss der Datensatz "1_Cleansing_Join" aus dem vorherigen Notebook reingeladen werden.

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
maindf = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/2_Data_for_Modeltraining.csv")
#maindf = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AI & ML/Projekt/1_Cleansing_Join_final.csv')

# Model Training & Evaluation

In [ ]:
X = maindf.drop(columns='adr')
y = maindf.adr

And split it into train and test sets:

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

We will test the following models and compare their performance:

*   Linear Regression (with and without regularization)
*   Support Vector Mechine (SVM)
*   Decision Tree
*   Gradient Boosting


Before we do that, let's again implement a small `evaluate` function for measuring the performance of our model:

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
def evaluate(reg, X_test, y_test):
  pred = reg.predict(X_test)
  print('R2:', r2_score(y_test, pred))
  print('MAE:', mean_absolute_error(y_test, pred))
  print('MSE:', mean_squared_error(y_test, pred))
  print('RMSE:', mean_squared_error(y_test, pred, squared=False))
  print('MAPE:', mean_absolute_percentage_error(y_test, pred))

## SVM

Next, we take a look at a support vector machine (SVM) model. Normally, SVMs are used for classification but they can also be applied to regression problems.

`sklearn` provides a SVM regression implementation which we can easily use:

In [ ]:
from sklearn.svm import LinearSVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
lsvr = make_pipeline(StandardScaler(),
                     LinearSVR(random_state=42, tol=1e-5))

In [ ]:
lsvr.fit(X_train, y_train)

In [ ]:
evaluate(lsvr, X_test, y_test)

The SVM achieves a pretty good result - significantly better than the regression models.

In [ ]:
y_lsvr_predict = lsvr.predict(X_test)

### **Hyperparameter Tuning**

C: da LinearSVC: andere Parameter wie 'learning_rate' nicht valid.

Die Fehlermeldung "TypeError: Parameter grid for parameter 'C' is not iterable or a distribution (value=1)" deutet darauf hin, dass es ein Problem mit dem angegebenen Parameterraum für den Parameter 'C' gibt. Der Wert '1' allein ist nicht als Parameterraum gültig.

Um das Problem zu beheben, kannst du entweder einen spezifischen Wert oder einen Parameterraum für 'C' angeben. Hier sind zwei Möglichkeiten:

from scipy.stats import uniform

param_grid = {
    'C': uniform(loc=0.1, scale=10),

    Die Wahl zwischen der Verwendung spezifischer Werte und der Verwendung eines Parameterraums hat Vor- und Nachteile, die ich im Folgenden erläutern werde:

Option 1: Spezifische Werte für 'C':
Vorteile:

Einfachheit: Die Verwendung spezifischer Werte ist einfach und erfordert keine weiteren Berechnungen.
Kontrolle: Du hast direkte Kontrolle über die ausgewählten Werte für 'C' und kannst gezielt bestimmte Werte testen.
Nachteile:

Möglicher Informationsverlust: Es besteht die Möglichkeit, dass der optimale Wert für 'C' nicht in den spezifischen Werten enthalten ist. Dies kann dazu führen, dass das Tuning nicht das bestmögliche Ergebnis erzielt.
Option 2: Parameterraum für 'C':
Vorteile:

Flexibilität: Durch die Verwendung eines Parameterraums kannst du einen breiteren Bereich von 'C'-Werten abdecken und somit eine größere Chance haben, den optimalen Wert zu finden.
Automatisierte Suche: Der Hyperparameter-Tuning-Algorithmus (z. B. RandomizedSearchCV) kann den Parameterraum automatisch durchsuchen und die besten Kombinationen finden.
Nachteile:

Komplexität: Die Definition eines Parameterraums erfordert möglicherweise zusätzliche Überlegungen und Kenntnisse über die Wertebereiche von 'C'. Die Auswahl eines geeigneten Parameterraums kann eine gewisse Erfahrung oder Experimentation erfordern.
Berechnungsaufwand: Ein breiterer Parameterraum kann zu einer erhöhten Rechenzeit führen, da der Hyperparameter-Tuning-Algorithmus mehr Kombinationen ausprobieren muss.
Es ist wichtig zu beachten, dass die Vor- und Nachteile von Optionen wie spezifischen Werten oder Parameterräumen stark vom Anwendungsfall und den Eigenschaften des Modells abhängen können. Es empfiehlt sich, verschiedene Optionen auszuprobieren und die Auswirkungen auf die Leistung des Modells zu beobachten, um die besten Hyperparameter-Tuning-Strategien zu ermitteln.



In [ ]:
from scipy.stats import loguniform

param_grid = {
    'C': [0.1, 1, 10],
    'class_weight': [None, 'balanced'],
    'dual': [True, False],
    'fit_intercept': [True, False],
    'intercept_scaling': loguniform(0.1, 10),
    'loss': ['hinge', 'squared_hinge'],
    'max_iter': [100, 500, 1000],
    'penalty': ['l1', 'l2'],
    'tol': loguniform(0.0001, 0.1),
    'verbose': [0, 1]
}

In [ ]:
ht_lsvr = RandomizedSearchCV(lsvr, param_grid, random_state=42, n_iter=10)

In [ ]:
ht_svc.fit(X_train, y_train)

Trotz Hyperparameter Tuning richtig schlechte Accuracy => nicht geeignet

In [ ]:
evaluate(ht_svc, X_test, y_test)

## Gradient Boosting

Let's move on to a gradient boosting model.

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
gb = make_pipeline(StandardScaler(),
                   GradientBoostingRegressor(random_state=42))

In [ ]:
gb.fit(X_train, y_train)

In [ ]:
evaluate(gb, X_test, y_test)

It achieves a very high performance!

In [ ]:
from sklearn.dummy import DummyRegressor

In [ ]:
dummy = DummyRegressor()

In [ ]:
dummy.fit(X_train, y_train)

In [ ]:
evaluate(dummy, X_test, y_test)

Instead of relying on a visual plot to determine the most important features, we can also utilize the `feature_importances_` attribute on the trained sklearn estimator:

In [ ]:
import numpy as np
feature_importance = gb.feature_importances_

# get indices sorted by importance
sorted_idx = np.argsort(feature_importance)

# generate range from 0 to the number of features
pos = np.arange(sorted_idx.shape[0])

# 'pos' acts as our value for the 'y' axis
plt.barh(pos, feature_importance[sorted_idx], align='center')

plt.yticks(pos, np.array(X.columns)[sorted_idx])
plt.title('Feature Importance (MDI)')
plt.show()

As expected, the feature importance we find here is similar to the one we found visually in the decision tree.

The model's performance appears to be quite promising so let's try to improve that further. For this purpose, we will perform some basic hyperparameter optimizations in a next step.

### Hyperparameter Tuning

We'll again be using the random search approach. As you can see from the `CV` suffix this is combined with cross-validation.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

Again, we first have to define our hyperparameter grid:

In [ ]:
# the algorithm will try out random values from the lists we provide
grid = {'learning_rate': [0.15, 0.1, 0.05, 0.01, 0.005, 0.001],
        'n_estimators': [100, 250, 500, 750, 1000, 1250, 1500, 1750],
        'max_depth': [2, 3, 4, 5, 6, 7]}

We can now start the tuning process - limited to 100 iterations (100 different hyperparameter combinations). Note that this process can take a while because sklearn will fit the model 5 times (folds) for each iteration resulting in 500 total fits:

In [ ]:
# setting n_jobs=-1 will ensure that sklearn uses all available cpu cores
# takes a loooong time (reduce n_iter to make it faster)
optimized_gb = RandomizedSearchCV(pipeline, param_grid, n_iter=100, n_jobs=-1, random_state=42, cv=5, verbose=1)
optimized_gb.fit(X_train, y_train)

Now that the tuning is done, we can retrieve the best hyperparamter combination as follows:

In [ ]:
optimized_gb.best_params_

And evaluate the model trained using those parameters:

In [ ]:
evaluate(optimized_gb, X_test, y_test)

We improved the performance of our model slightly - not a huge improvement, but better than nothing!

The gradient boosting model (in fact, all ML models) might suffer from overfitting where they memorize the data instead of learning the underlying patterns. One way of detecting this for a gradient boosting model is plotting the train and test error against the number of iterations (i.e. how long the model was trained for).

Let's create this plot here:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# get best xgb model (after hyperparam tuning)
best_gb = optimized_xgb.best_estimator_
# staged_predict returns the error after each stage in the model
predictions = best_gb.staged_predict(X_test)
# how many estimators were used for the model
n_estimators = optimized_gb.best_params_['n_estimators']

# calculate deviance (error) for all examples in test set
test_score = np.zeros((n_estimators,), dtype=np.float64)
for i, y_pred in enumerate(predictions):
    test_score[i] = best_gb.loss_(y_test, y_pred)

Now we can plot the test loss against the training loss which is already stored in the `train_score_` attribute.

In [ ]:
fig = plt.figure(figsize=(6, 6))
plt.subplot(1, 1, 1)
plt.title('Deviance')
plt.plot(np.arange(n_estimators) + 1, best_xgb.train_score_, 'b-',
         label='Training Set Deviance')
plt.plot(np.arange(n_estimators) + 1, test_score, 'r-',
         label='Test Set Deviance')
plt.legend(loc='upper right')
plt.xlabel('Boosting Iterations')
plt.ylabel('Deviance')
fig.tight_layout()
plt.show()

## **XGB Regression**

In [ ]:
# plot decision tree
from numpy import loadtxt
from xgboost import XGBRegressor
from xgboost import plot_tree
import matplotlib.pyplot as plt

# fit model no training data
xgb = make_pipeline(StandardScaler(),
                    XGBRegressor())
xgb.fit(X_train, y_train)

In [ ]:
evaluate(xgb, X_train, y_train)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# the algorithm will try out random values from the lists we provide
grid = {
    'xgbregressor__n_estimators': [100, 250, 500, 750, 1000, 1250, 1500, 1750],
    'xgbregressor__max_depth': [2, 3, 4, 5, 6, 7],
    'xgbregressor__learning_rate': [0.15, 0.1, 0.05, 0.01, 0.005, 0.001]
}

We can now start the tuning process - limited to 100 iterations (100 different hyperparameter combinations). Note that this process can take a while because sklearn will fit the model 5 times (folds) for each iteration resulting in 500 total fits:

In [ ]:
# setting n_jobs=-1 will ensure that sklearn uses all available cpu cores
# takes a loooong time (reduce n_iter to make it faster)
optimized_xgb = RandomizedSearchCV(xgb, grid, n_iter=100, n_jobs=-1, random_state=42, verbose=3)
optimized_xgb.fit(X_train, y_train)

Now that the tuning is done, we can retrieve the best hyperparamter combination as follows:

In [ ]:
optimized_xgb.best_params_

And evaluate the model trained using those parameters:

In [ ]:
evaluate(optimized_xgb, X_test, y_test)

We can see that there is not a big difference between training and test set deviance which means that our model is not suffering from overfitting (at least not heavily). However, we might choose to stop training after around iteration 20 as the additional iterations only contribute very little to the overall model performance. That being said, we are only looking at an individual hyperparam here so this might not be the true optimal parameter value.

# Lineare Regression

In [ ]:
from sklearn.linear_model import LinearRegression
linear = LinearRegression()

In [ ]:
from sklearn.pipeline import make_pipeline

linear_scaled = make_pipeline(scaler, linear)

In [ ]:
linear_scaled.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearregression', LinearRegression())])

In [ ]:
evaluate_lin_reg(linear_scaled, X_test, y_test)

R2: 0.42282568506105156
MAE: 27.16709864849837
MSE: 1397.8925400717947
RMSE: 37.38840114356048
MAPE: 5558511303644893.0


In [ ]:
from sklearn.linear_model import RidgeCV

In [ ]:
import numpy as np

In [ ]:
n_alphas = 200
alphaValues = np.logspace(-10, -1, n_alphas)

linear_ridge = make_pipeline(scaler, RidgeCV(alphas=alphaValues))

In [ ]:
linear_ridge.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('ridgecv',
                 RidgeCV(alphas=array([1.00000000e-10, 1.10975250e-10, 1.23155060e-10, 1.36671636e-10,
       1.51671689e-10, 1.68318035e-10, 1.86791360e-10, 2.07292178e-10,
       2.30043012e-10, 2.55290807e-10, 2.83309610e-10, 3.14403547e-10,
       3.48910121e-10, 3.87203878e-10, 4.29700470e-10, 4.76861170e-10,
       5.29197874e-10, 5.87278661...
       9.11588830e-03, 1.01163798e-02, 1.12266777e-02, 1.24588336e-02,
       1.38262217e-02, 1.53436841e-02, 1.70276917e-02, 1.88965234e-02,
       2.09704640e-02, 2.32720248e-02, 2.58261876e-02, 2.86606762e-02,
       3.18062569e-02, 3.52970730e-02, 3.91710149e-02, 4.34701316e-02,
       4.82410870e-02, 5.35356668e-02, 5.94113398e-02, 6.59318827e-02,
       7.31680714e-02, 8.11984499e-02, 9.01101825e-02, 1.00000000e-01])))])

In [ ]:
linear_ridge[1].alpha_

0.1

In [ ]:
evaluate_lin_reg(linear_ridge, X_test,y_test)

R2: 0.4228256738331103
MAE: 27.167096968333425
MSE: 1397.8925672654082
RMSE: 37.388401507224245
MAPE: 5558511909652077.0


In [ ]:
maindf.adr.mean()

102.46612039949844